# CSGY-6513 Big Data Final Project
In this notebook, I implement the Jaccard Similarity model with Min-Hash and Locality Sensitive Hashing on Kaggle Dataset.

## 1. Setting up PyMongo, Store the Data into MongoDB.

In [1]:
from pymongo import MongoClient
client = MongoClient("mongo-csgy-6513-fall.db",
                     username='as15106',
                     password='as15106',
                     authSource='as15106_db')
db = client.as15106_db

In [2]:
import pymongo
import re
import numpy as np
import time

In [3]:
def show(cursor):
    for c in cursor:
        print(c)

In [4]:
import csv
data = []
with open('Combined_News_DJIA.csv', 'r') as f:
    csvfile = csv.reader(f)
    for i, row in enumerate(csvfile):
        if i == 0:
            header = row
        else:
            doc = {}
            news = {}
            if i <= 1789:
                doc["category"] = "train"
            else:
                doc["category"] = "test"
            for j in range(len(row)):
                if header[j][:3] != "Top":
                    doc[header[j]] = row[j]
                else:
                    news[header[j]] = {"headline": row[j]}
            doc["news"] = news
            data.append(doc)
db.data.drop()
db.data.insert_many(data)

In [5]:
db.data.count_documents({})

1989

In [6]:
db.data.count_documents({"category": "train"})

1789

In [7]:
db.data.count_documents({"category": "test"})

200

In [8]:
cursor = db.data.find({}).limit(1)
show(cursor)

{'_id': ObjectId('639e0288587af75749150ef5'), 'category': 'train', 'Date': '2008-08-08', 'Label': '0', 'news': {'Top1': {'headline': 'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war"'}, 'Top2': {'headline': "b'BREAKING: Musharraf to be impeached.'"}, 'Top3': {'headline': "b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'"}, 'Top4': {'headline': "b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'"}, 'Top5': {'headline': 'b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing"'}, 'Top6': {'headline': "b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.'"}, 'Top7': {'headline': 'b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side"'}, 'Top8': {'headline': 'b"The \'enemy combatent\' trials

## 2. Preprocessing, Encoding into Binary Vector Representation.

In [9]:
# A function for lowercasing, removing unnecessary letters, and splitting text into a list of words.
def cleaning(string):
    if string[0] == 'b':
        string = string[1:]
    string = re.sub("[^0-9a-z]", " ", string.lower()).split()
    return string

In [10]:
# Create a document for word counting.
db.data.insert_one({"category": "other", 
                    "name": "count", 
                    "count": {}})
count_id = db.data.find_one({"category": "other", "name": "count"})
count_id = count_id["_id"]

In [11]:
last = time.time()
# Preprocess the document one by one.
for i, doc in enumerate(db.data.find({"$or": [{"category": "train"}, {"category": "test"}]})):
    words = {}
    for rank in doc["news"]:
        # Each news headline is cleaned and stored.
        cleaned = cleaning(doc["news"][rank]["headline"])
        db.data.update_one({"_id": doc["_id"]}, {"$set": {"news."+rank+".parsed": cleaned}})
        # To focus on frequent words, perform the word counting.
        if doc["category"] == "train":
            for word in cleaned:
                if word in words.keys():
                    words[word] += 1
                else:
                    words[word] = 1
    # Save the word counting result to our word count document.
    for word, n in words.items():
        if db.data.count_documents({"_id": count_id, "count."+word: {"$exists": True}}) == 0:
            db.data.update_one({"_id": count_id}, {"$set": {"count."+word: n}})
        else:
            db.data.update_one({"_id": count_id}, {"$inc": {"count."+word: n}})
    if time.time() - last > 600:
        print(f"Processing {i}-th data...")
        last = time.time()

Processing 442-th data...
Processing 718-th data...
Processing 977-th data...
Processing 1238-th data...
Processing 1480-th data...
Processing 1742-th data...


In [12]:
# Select the top 1000 frequent words.
freq = []
doc = db.data.find_one({"_id": count_id})
for word in doc["count"]:
    freq.append(doc["count"][word])
freq.sort(reverse=True)
threshold = freq[1000]

In [13]:
# Create a document for word-index mapping.
db.data.insert_one({"category": "other", 
                    "name": "word2index", 
                    "word2index": {}})
word2index_id = db.data.find_one({"category": "other", "name": "count"})
word2index_id = word2index_id["_id"]

index = 0
for word in doc["count"]:
    if doc["count"][word] > threshold:
        db.data.update_one({"_id": word2index_id}, {"$set": {"word2index."+word: index}})
        index += 1

In [14]:
# A function to convert text into a binary vector.
# Instead of keeping the whole vector, we keep the index of entries having 1.
def encode(words):
    ret = set()
    for word in words:
        if db.data.count_documents({"_id": word2index_id, "word2index."+word: {"$exists": True}}) > 0:
            ret.add(db.data.find_one({"_id": word2index_id}, 
                                     projection={"word2index."+word})["word2index"][word])
    return list(ret)

# A function for Min-Hash (A function used for estimating Jaccard Similarity).
def minhash(lst, a, b, prime):
    arr = np.array(lst)
    hash_arr = (a[:, None] * arr[None, :] + b[:, None]) % prime
    return np.min(hash_arr, axis=1).tolist()

# A function for computing Jaccard Similarity.
def Jaccard(lst1, lst2):
    set1, set2 = set(lst1), set(lst2)
    return len(set1 & set2) / len(set1 | set2)

# A function for estimating Jaccard Similarity from Min-Hash.
def JaccardEstimate(lst1, lst2, k):
    return sum(np.array(lst1)==np.array(lst2)) / k

# Parameters.
# k: Number of hash functions in Min-Hash.
# prime, a, b: Our hash functions.
k = 120
prime = 8191
a = np.random.randint(1, prime, k)
b = np.random.randint(0, prime, k)

In [15]:
# Encoding texts into binary vectors, and store them in the MongoDB.
last = time.time()
for i, doc in enumerate(db.data.find({"$or": [{"category": "train"}, {"category": "test"}]})):
    words = []
    for rank in doc["news"]:
        words += doc["news"][rank]["parsed"]
    lst = encode(words)
    db.data.update_one({"_id": doc["_id"]}, {"$set": {"birep": lst, "hash": minhash(lst, a, b, prime)}})
    if time.time() - last > 600:
        print(f"Processing {i}-th data...")
        last = time.time()

Processing 592-th data...
Processing 1152-th data...
Processing 1673-th data...


In [16]:
cursor = db.data.find({}).limit(1)
show(cursor)

{'_id': ObjectId('639e0288587af75749150ef5'), 'category': 'train', 'Date': '2008-08-08', 'Label': '0', 'news': {'Top1': {'headline': 'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war"', 'parsed': ['georgia', 'downs', 'two', 'russian', 'warplanes', 'as', 'countries', 'move', 'to', 'brink', 'of', 'war']}, 'Top2': {'headline': "b'BREAKING: Musharraf to be impeached.'", 'parsed': ['breaking', 'musharraf', 'to', 'be', 'impeached']}, 'Top3': {'headline': "b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'", 'parsed': ['russia', 'today', 'columns', 'of', 'troops', 'roll', 'into', 'south', 'ossetia', 'footage', 'from', 'fighting', 'youtube']}, 'Top4': {'headline': "b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'", 'parsed': ['russian', 'tanks', 'are', 'moving', 'towards', 'the', 'capital', 'of', 'south', 'ossetia', 'which', 'has', 'repo

## 3. Prediction with Locality Sensitive Hashing.

In [17]:
# Parameters.
# r: Number of bands in LSH.
# prime, a, b: Our hash functions.
r = 2
prime = 35027
a_c = np.random.randint(1, prime, r)
b_c = np.random.randint(0, prime, r)
# A function for Min-Hash (A function used for locality sensitive hashing).
def MinHash(lst, a, b, prime):
    arr = np.array(lst)
    hash_arr = (a[:, None] * arr[None, :] + b[:, None]) % prime
    return np.min(hash_arr, axis=1)

# Parameters.
# t: Number of tables in LSH.
# prime, a, b: Our hash functions.
t = 100
a_g = np.random.randint(1, prime, (t, r))
b_g = np.random.randint(0, prime, t)
# A function for finding bins to store each data.
def LSH(arr, a, b, prime):
    hash_arr = (np.dot(a, arr) + b) % prime
    return hash_arr

In [18]:
# Create a document for LSH.
last = time.time()
db.data.insert_one({"category": "other", 
                    "name": "lsh", 
                    "lsh": {}})
lsh_id = db.data.find_one({"category": "other", "name": "lsh"})
lsh_id = lsh_id["_id"]

for i, doc in enumerate(db.data.find({"category": "train"})):
    bins = LSH(MinHash(doc["birep"], a_c, b_c, prime), a_g, b_g, prime)
    for b in bins:
        if db.data.count_documents({"_id": lsh_id, "lsh."+str(b): {"$exists": True}}) == 0:
            db.data.update_one({"_id": lsh_id}, {"$set": {"count."+str(b): [doc["_id"]]}})
        else:
            db.data.update_one({"_id": lsh_id}, {"$push": {"count."+str(b): [doc["_id"]]}})
    if time.time() - last > 600:
        print(f"Processing {i}-th data...")
        last = time.time()

Processing 1571-th data...


In [34]:
# Make a prediction.
prediction = []
num_query = []
doc = db.data.find_one({"_id": lsh_id})
lsh_map = doc["count"]
for i, doc_test in enumerate(db.data.find({"category": "test"})):
    # Find bins to check.
    bins = LSH(MinHash(doc_test["birep"], a_c, b_c, prime), a_g, b_g, prime)
    # Find training data ID to compare the similarity.
    candidate = set()
    for b in bins:
        if str(b) in lsh_map.keys():
            for _id in lsh_map[str(b)]:
                candidate.add(_id)
    # num_query is to track how many queries we make.
    num_query.append(len(candidate))
    # Searching a binary vector with the highest similarity.
    best_id, best_js = -1, -1
    for doc_train in db.data.find({"_id": {"$in": list(candidate)}}):
        js = Jaccard(doc_test["birep"], doc_train["birep"])
        # Instead of computing the exact Jaccard similarity, we can use the following line to make calculation faster with the use of Min-Hash.
        # js = JaccardEstimate(doc_test["hash"], doc_train["hash"])
        if best_js < js:
            best_js = js
            best_id = doc_train["_id"]
    best_doc = db.data.find_one({"_id": best_id})
    if best_doc["Label"] == doc_test["Label"]:
        prediction.append(1)
    else:
        prediction.append(0)

In [35]:
# Finally, check the accuracy.
acc = sum(prediction) / len(prediction)
print(f"Test Accuracy: {acc}")

Test Accuracy: 0.545


In [36]:
# Check the number of queries on average.
ave_query = np.mean(np.array(num_query))
print(f"Number of Queries: {ave_query}")

Number of Queries: 7.55
